### Create Model for Participant 1

In [2]:
import os
import numpy as np

In [3]:
# load all image vectors
path_to_features = "/home/aditis/decodingEEG/DecodeEEG/data/feature_vectors"

# Initialize an empty list to store the loaded arrays
data = []

classes = os.listdir(path_to_features+"/training/")

for clas in classes:
    feature_path  = path_to_features + "/training/" + clas
    files = os.listdir(feature_path)
    for file in files:
        if file.endswith('.npy'):  # Ensure it's a .npy file
            file_path = feature_path +"/"+ file
            array = np.load(file_path)  # Load the .npy file
            data.append(array)         # Append to the list

    
# Convert the list of arrays into a matrix
feature_matrix = np.vstack(data)  # Stack arrays vertically

# Verify the shape
print(feature_matrix.shape)  # Output: (16540, 512)



(16540, 512, 7, 7)


In [51]:
# model architecture
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.utils.vis_utils import plot_model

# Build the CNN model
model = models.Sequential()

# Input layer
model.add(layers.InputLayer(input_shape=(17, 26, 26)))

# Convolutional layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))  # 32 filters, kernel size 3x3
model.add(layers.MaxPooling2D((2, 2)))  # Max pooling (2x2)

model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))  # 64 filters, kernel size 3x3
model.add(layers.MaxPooling2D((2, 2)))  # Max pooling (2x2)

model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))  # 128 filters, kernel size 3x3
model.add(layers.MaxPooling2D((2, 2)))  # Max pooling (2x2)

# Flatten the output of the last convolutional layer
model.add(layers.Flatten())

# Dense layers to map to 512-dimensional feature vector
model.add(layers.Dense(512, activation='relu'))  # Dense layer with 512 neurons
model.add(layers.Dropout(0.3))  # Dropout for regularization
model.add(layers.Dense(512, activation='linear'))  # Output layer with 512 neurons (regression task)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True) 

# Summary of the model architecture
model.summary()



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 17, 26, 32)        7520      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 8, 13, 32)        0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 8, 13, 64)         18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 4, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 4, 6, 128)         73856     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 2, 3, 128)       

In [32]:
# function to convert eeg to spectrogram (17, 401, 75)
from scipy.signal import ShortTimeFFT
from scipy.signal.windows import gaussian

def eeg_to_spectrogram(eeg):
    """
    Converts a single EEG signal (17x100) into its spectrogram.
    """
    g_std = 12  # standard deviation for Gaussian window in samples
    M = 50 # window size
    win = gaussian(M, std=g_std, sym=True)  # symmetric Gaussian window
    T_x, N = 1 / 100, 100  # Sampling rate: 100 Hz, signal length: 100 samples
    SFT = ShortTimeFFT(win, hop=2, fs=1/T_x,scale_to='psd')

    # Calculate Spectrogram
    Sx2 = SFT.spectrogram(eeg)  # shape: (17, ?, ?)
    truncate_index = int(M/2)
    print(truncate_index-1)
    print(Sx2.shape[-1]-truncate_index)
    return Sx2[:,:,truncate_index-1:Sx2.shape[-1]-truncate_index]

In [33]:
# Load eeg data
path_to_eeg = "/home/aditis/decodingEEG/DecodeEEG/data/PreprocessedEEG/sub-01"
data = np.load(path_to_eeg+'/preprocessed_eeg_training.npy', allow_pickle=True).item()
eeg_data = data["preprocessed_eeg_data"]
print(eeg_data.shape)

spec = eeg_to_spectrogram(eeg_data[0,0,:,:])
spec.shape

(16540, 4, 17, 100)
24
50


(17, 26, 26)

In [47]:
import h5py
path_to_spec = "/home/aditis/decodingEEG/DecodeEEG/data/spectrograms/sub-01/"
with h5py.File(path_to_spec+'spectrograms_train.h5', 'r') as f: 
    # Access the dataset
    spectrograms = f['spectrograms']  # This is a reference to the dataset
    print(f"Spectrogram dataset shape: {spectrograms.shape}")  # Example: (N, M, 17, 26, 26)


Spectrogram dataset shape: (16540, 4, 17, 26, 26)


In [45]:
def train_batch_generator(path_to_spectrograms, feature_matrix, batch_size=1654):
    """
    Generator for training data batches.
    Input : EEG data of one subject (16540 x 4 x 17 x 26 x 26), feature matrix of all images (16540 x 512)
    Output : Batches of spectrograms (1654 x 17 x 401 x 75) and feature vectors (1654 x 512)
    """
    with h5py.File(path_to_spectrograms+'spectrograms_train.h5', 'r') as f: 
        # Access the dataset
        spectrograms = f['spectrograms']  # This is a reference to the dataset
        num_samples = spectrograms.shape[0]

        while True:  # Infinite loop to yield batches
            for i in range(spectrograms.shape[1] - 1):  # Iterate over EEG sets (3 sets)
                spec  = spectrograms[:, i, :, :]
                for k in range(0, num_samples, batch_size):  # Iterate over chunks
                    batch_end = min(k + batch_size, num_samples)  # Ensure no overflow
                    spectro = spec[k:batch_end,:,:]
                    features = feature_matrix[k:batch_end, :]
                    yield (spectro, features)  # Yield the batch



def validation_batch_generator(path_to_spectrograms, feature_matrix, batch_size=1654):
    """
    Generator for validation data batches.
    Input : EEG data of one subject (16540 x 4 x 17 x 26 x 26), feature matrix of all images (16540 x 512)
    Output : Batches of spectrograms (1654 x 17 x 401 x 75) and feature vectors (1654 x 512)
    """
    with h5py.File(path_to_spectrograms+'spectrograms_train.h5', 'r') as f: 
        # Access the dataset
        spectrograms = f['spectrograms']  # This is a reference to the dataset
        num_samples = spectrograms.shape[0]
        spec  = spectrograms[:, 3, :, :]
        while True:  # Infinite loop to yield batches
            for k in range(0, num_samples, batch_size):  # Iterate over chunks
                batch_end = min(k + batch_size, num_samples)  # Ensure no overflow
                spectro = spec[k:batch_end,:,:]
                features = feature_matrix[k:batch_end, :]
                yield (spectro, features)  # Yield the batch



In [55]:
# Define generator for spectrograms
def test_batch_generator(path_to_spec, num_samples, batch_size=20):
    with h5py.File(path_to_spec+'spectrograms_test.h5', 'r') as f: 
        spectrograms = f['spectrograms'] 
        for i in range(0, num_samples, batch_size):
            spectro_batch = spectrograms[i:i+batch_size, :, :, :]
            print(spectro_batch.shape)
            #yield spectro_batch

test_batch_generator("/home/aditis/decodingEEG/DecodeEEG/data/spectrograms/sub-01/", 200)

(20, 80, 17, 26, 26)
(20, 80, 17, 26, 26)
(20, 80, 17, 26, 26)
(20, 80, 17, 26, 26)
(20, 80, 17, 26, 26)
(20, 80, 17, 26, 26)
(20, 80, 17, 26, 26)
(20, 80, 17, 26, 26)
(20, 80, 17, 26, 26)
(20, 80, 17, 26, 26)


In [52]:
# Set parameters
batch_size = 1654
num_samples = eeg_data.shape[0]  # Total samples in dataset
steps_per_epoch = num_samples * 3 // batch_size  # 3 EEG sets for training
validation_steps = num_samples // batch_size    # 1 EEG set for validation
nb_epoch = 10
path_to_specs = "/home/aditis/decodingEEG/DecodeEEG/data/spectrograms/sub-01/"
# Train the model
model.fit(
    train_batch_generator(path_to_specs, feature_matrix, batch_size=batch_size),
    epochs=nb_epoch,
    steps_per_epoch=steps_per_epoch,
    verbose=1,
    validation_data=validation_batch_generator(path_to_specs, feature_matrix, batch_size=batch_size),
    validation_steps=validation_steps
)

# Save the model
model.save("/home/aditis/decodingEEG/DecodeEEG/data/simple_model_weights/eeg_to_feature_vector_model.h5")


Epoch 1/10
30/30 [==============================] - 6s 183ms/step - loss: 2.5400 - mae: 0.7296 - val_loss: 1.1464 - val_mae: 0.6857
Epoch 2/10
30/30 [==============================] - 5s 160ms/step - loss: 0.9743 - mae: 0.6746 - val_loss: 0.7901 - val_mae: 0.6427
Epoch 3/10
30/30 [==============================] - 5s 161ms/step - loss: 0.8131 - mae: 0.6519 - val_loss: 0.7882 - val_mae: 0.6474
Epoch 4/10
30/30 [==============================] - 5s 159ms/step - loss: 0.8061 - mae: 0.6547 - val_loss: 0.7877 - val_mae: 0.6483
Epoch 5/10
30/30 [==============================] - 5s 158ms/step - loss: 0.8024 - mae: 0.6537 - val_loss: 0.7877 - val_mae: 0.6492
Epoch 6/10
30/30 [==============================] - 5s 160ms/step - loss: 0.8001 - mae: 0.6528 - val_loss: 0.7877 - val_mae: 0.6512
Epoch 7/10
30/30 [==============================] - 5s 159ms/step - loss: 0.7988 - mae: 0.6524 - val_loss: 0.7878 - val_mae: 0.6494
Epoch 8/10
30/30 [==============================] - 5s 160ms/step - loss: 0.